In [1]:
import pandas as pd
from sklearn import ensemble, cross_validation, metrics
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold
from sklearn.model_selection import GridSearchCV

D:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
model_data = pd.read_csv('./model_data.csv', encoding='utf-8')
model_data.head()

,LOCAL_CUR_SAV_SLOPE,LOCAL_CUR_MON_AVG_BAL,LOCAL_OVEONEYR_FF_MON_AVG_BAL,LOCAL_FIX_MON_AVG_BAL,LOCAL_FIX_MON_AVG_BAL_PROP,LOCAL_BELONEYR_FF_SLOPE,LOCAL_BELONEYR_FF_MON_AVG_BAL,LOCAL_OVEONEYR_FF_SLOPE,LOCAL_SAV_SLOPE,LOCAL_SAV_CUR_ALL_BAL,...,kid6_10_nan,kid11_15_U,kid11_15_Y,kid11_15_nan,kid16_17_U,kid16_17_Y,kid16_17_nan,car_buy_New,car_buy_UNKNOWN,car_buy_nan
0,0.333333,0.000000,0.009068,0.001157,1.000000,0.344437,0.0,0.333284,0.333196,9.277653e-04,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,0.668564,0.002648,0.010078,0.001286,0.735565,0.344437,0.0,0.293169,0.345716,7.225051e-04,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,0.333521,0.000002,0.000000,0.000000,0.000000,0.344437,0.0,0.344442,0.344545,4.032385e-07,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,0.149696,0.003858,0.000000,0.000000,0.000000,0.344437,0.0,0.344442,0.154644,7.529284e-04,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.333429,0.000005,0.000000,0.000000,0.000000,0.344437,0.0,0.344442,0.344450,7.852062e-07,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [3]:
all_features = list(model_data.columns)
all_features.remove('CHURN_CUST_IND')

In [4]:
X_train,  X_test, y_train, y_test = train_test_split(model_data[all_features], model_data['CHURN_CUST_IND'], test_size=0.3, random_state=1)

In [5]:
print('y_train_count: ', y_train.value_counts())
print('y_test_count: ', y_test.value_counts())

y_train_count:  0.0    10851
1.0     1217
Name: CHURN_CUST_IND, dtype: int64
y_test_count:  0.0    4649
1.0     524
Name: CHURN_CUST_IND, dtype: int64


In [6]:
# 使用GBDT模块的默认参数进行训练
gbm0 = GradientBoostingClassifier(random_state=10)
gbm0.fit(X_train,y_train)
y_pred = gbm0.predict(X_test)
y_predprob = gbm0.predict_proba(X_test)[:,1]
print("Accuracy : %.4g" % metrics.accuracy_score(y_test, y_pred))
print ("AUC Score (Testing): %f" % metrics.roc_auc_score(y_test, y_predprob))

Accuracy : 0.9115
AUC Score (Testing): 0.855843
